In [1]:
# Base Libraries
import re
import time 
import numpy as np
import pandas as pd
import statistics
import warnings
from sklearn.metrics import accuracy_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


warnings.filterwarnings(action='ignore')

# Library for Google Colab
from google.colab import drive

##**Accident Severity Prediction Using Classification**

Accident severity has been predicted using traditional methods of statistics in the past, such as logit algos, multinomial logit and logistic regression. However, these measures have been unable to predict non-linear relationships between independent factors that affect the causality of a road accident, thereby predicting inaccurate severity measures. The primary objective of this project is building a predictive model to predict the severity of accident in UK.

##**Table of Contents**
 
1.   Introduction: business objective
2.   Feature selection
3.   Feature scaling
4.   Baseline method
5.   Hyperparameter tuning
6.   Model evaluation
7.   Conclusion: key findings, possible future improvements.

##**Business objective:**
Most car accident in the UK occur on high speed motorways in the countryside that are far from any hospital or emergency services reach. As a consequence, many chronic injuries are sustained, along with loss of life. Most injuries incured in car accidents can be prevented if proper care relief is provided at the right time. In times of multiple car accident crashes in the vicinity of the nearest emergency service, the aid has only a limited number of resources, which cannot practically be disperesed to each accident site. A situation like this requires priority based response.

The objective of this project is to predict accident severity accurately based on predictor variables that can be obtained from the site of crash. The severity rating can then be used to prioritise dispatch of response team and organize first aid resources accordingly. Furthermore, this report discusses the mapping of accident hotspots to encourage resource allocation in specific areas.



In [2]:
# Mount Google Drive on kernel
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/Colab Notebooks/Content"

Mounted at /content/gdrive


In [3]:
# Loading the datasets into dataframes
x_accident_train = pd.read_csv("gdrive/My Drive/Colab Notebooks/Content/x_accident_train.csv",
                               index_col=0)
x_accident_test = pd.read_csv("gdrive/My Drive/Colab Notebooks/Content/x_accident_test.csv", 
                              index_col=0)
y_accident_train = pd.read_csv("gdrive/My Drive/Colab Notebooks/Content/y_accident_train.csv", 
                               index_col=0)
y_accident_test = pd.read_csv("gdrive/My Drive/Colab Notebooks/Content/y_accident_test.csv", 
                              index_col=0)

In [4]:
y_accident_train.head()
y_trainset = y_accident_train["accident_severity"]

In [5]:
y_accident_test.head()
y_testset = y_accident_test["accident_severity"]

##**Checking Correlation with features obtained after Onehotencoding:**
First generating a correlation matrix by using predictore variables against target variables. Then identifying features whose correlation value are between 1 and -1. 

In [6]:
correlation_matrix = x_accident_train.corrwith(y_trainset, axis=0, drop=False)
print(correlation_matrix.sort_values(ascending=False))

light_conditions_6                    0.011559
age_band_of_casualty_9                0.011352
casualty_severity_2                   0.009372
age_band_of_driver_9                  0.007953
first_point_of_impact_3               0.007600
                                        ...   
vehicle_location_restricted_lane_4   -0.009365
junction_location_5                  -0.010319
casualty_severity_3                  -0.010669
light_conditions_4                   -0.011245
age_band_of_driver_1                 -0.015172
Length: 118, dtype: float64


In [7]:
column_names = list(x_accident_train.columns)
print(column_names)

['speed_limit', 'age_of_vehicle', 'engine_capacity_cc', 'latitude', 'longitude', 'first_point_of_impact_0', 'first_point_of_impact_1', 'first_point_of_impact_2', 'first_point_of_impact_3', 'first_point_of_impact_4', 'trunk_road_flag_1', 'trunk_road_flag_2', 'urban_or_rural_area_1', 'urban_or_rural_area_2', 'age_band_of_casualty_1', 'age_band_of_casualty_2', 'age_band_of_casualty_3', 'age_band_of_casualty_4', 'age_band_of_casualty_5', 'age_band_of_casualty_6', 'age_band_of_casualty_7', 'age_band_of_casualty_8', 'age_band_of_casualty_9', 'age_band_of_casualty_10', 'age_band_of_casualty_11', 'vehicle_leaving_carriageway_0', 'vehicle_leaving_carriageway_1', 'vehicle_leaving_carriageway_2', 'vehicle_leaving_carriageway_3', 'vehicle_leaving_carriageway_4', 'vehicle_leaving_carriageway_5', 'vehicle_leaving_carriageway_6', 'vehicle_leaving_carriageway_7', 'vehicle_leaving_carriageway_8', 'vehicle_leaving_carriageway_9', 'light_conditions_1', 'light_conditions_4', 'light_conditions_5', 'light_c

##**Selecting the features based on correlation value:**
Once the features of non-interest are identified using correlation and a bit of common sense. They are dropped to improve model fitting. This can be considered as dimensionality reduction, cause removing unwanted features can improve the performance of a model.

In [8]:
x_accident_train = x_accident_train.drop(['latitude', 'longitude', 'speed_limit', 
                                          'age_of_vehicle', 'engine_capacity_cc', 
                                          'trunk_road_flag_1', 'trunk_road_flag_2', 
                                          'age_band_of_casualty_1', 
                                          'age_band_of_casualty_2', 
                                          'age_band_of_casualty_3', 
                                          'age_band_of_casualty_4', 
                                          'age_band_of_casualty_5', 
                                          'age_band_of_casualty_6', 
                                          'age_band_of_casualty_7', 
                                          'age_band_of_casualty_8', 
                                          'age_band_of_casualty_9', 
                                          'age_band_of_casualty_10', 
                                          'age_band_of_casualty_11', 
                                          'vehicle_leaving_carriageway_1', 
                                          'vehicle_leaving_carriageway_2', 
                                          'vehicle_leaving_carriageway_3', 
                                          'vehicle_leaving_carriageway_4', 
                                          'vehicle_leaving_carriageway_5', 
                                          'vehicle_leaving_carriageway_6', 
                                          'vehicle_leaving_carriageway_7', 
                                          'vehicle_leaving_carriageway_8', 
                                          'vehicle_leaving_carriageway_9', 
                                          'age_band_of_driver_3', 
                                          'age_band_of_driver_4', 
                                          'age_band_of_driver_5', 
                                          'age_band_of_driver_6', 
                                          'age_band_of_driver_7', 
                                          'age_band_of_driver_8', 
                                          'age_band_of_driver_9', 
                                          'age_band_of_driver_10', 
                                          'age_band_of_driver_11', 
                                          'second_road_class_1', 
                                          'second_road_class_2', 
                                          'second_road_class_3', 
                                          'second_road_class_4', 
                                          'second_road_class_5', 
                                          'second_road_class_6'], axis=1)
x_accident_train.shape

(64674, 76)

In [9]:
x_accident_test = x_accident_test.drop(['latitude', 'longitude', 'speed_limit', 
                                          'age_of_vehicle', 'engine_capacity_cc', 
                                          'trunk_road_flag_1', 'trunk_road_flag_2', 
                                          'age_band_of_casualty_1', 
                                          'age_band_of_casualty_2', 
                                          'age_band_of_casualty_3', 
                                          'age_band_of_casualty_4', 
                                          'age_band_of_casualty_5', 
                                          'age_band_of_casualty_6', 
                                          'age_band_of_casualty_7', 
                                          'age_band_of_casualty_8', 
                                          'age_band_of_casualty_9', 
                                          'age_band_of_casualty_10', 
                                          'age_band_of_casualty_11', 
                                          'vehicle_leaving_carriageway_1', 
                                          'vehicle_leaving_carriageway_2', 
                                          'vehicle_leaving_carriageway_3', 
                                          'vehicle_leaving_carriageway_4', 
                                          'vehicle_leaving_carriageway_5', 
                                          'vehicle_leaving_carriageway_6', 
                                          'vehicle_leaving_carriageway_7', 
                                          'vehicle_leaving_carriageway_8', 
                                          'vehicle_leaving_carriageway_9', 
                                          'age_band_of_driver_3', 
                                          'age_band_of_driver_4', 
                                          'age_band_of_driver_5', 
                                          'age_band_of_driver_6', 
                                          'age_band_of_driver_7', 
                                          'age_band_of_driver_8', 
                                          'age_band_of_driver_9', 
                                          'age_band_of_driver_10', 
                                          'age_band_of_driver_11', 
                                          'second_road_class_1', 
                                          'second_road_class_2', 
                                          'second_road_class_3', 
                                          'second_road_class_4', 
                                          'second_road_class_5', 
                                          'second_road_class_6'], axis=1)
x_accident_test.shape

(27718, 76)

##**Feature Scaling using Standard scaler:**
Normalize the range of independent variables or features of data.

In [10]:
scaler = StandardScaler()

# fit_transform returns a NumPy array, so we need to put it back 
# into a Pandas dataframe
scaled_vals = scaler.fit_transform(x_accident_train)
x_trainset = pd.DataFrame(scaled_vals, columns=x_accident_train.columns)

# inspect the data
x_trainset.head()

,first_point_of_impact_0,first_point_of_impact_1,first_point_of_impact_2,first_point_of_impact_3,first_point_of_impact_4,urban_or_rural_area_1,urban_or_rural_area_2,vehicle_leaving_carriageway_0,light_conditions_1,light_conditions_4,...,junction_location_9,sex_of_casualty_1,sex_of_casualty_2,sex_of_driver_1,sex_of_driver_2,did_police_officer_attend_scene_of_accident_1,did_police_officer_attend_scene_of_accident_2,casualty_severity_1,casualty_severity_2,casualty_severity_3
0,-0.202145,-1.052266,-0.449204,-0.39983,2.592023,-1.173652,1.173652,0.495507,-1.523049,-0.505025,...,-0.09934,-1.273240,1.273240,-1.499439,1.499439,0.606256,-0.606256,-0.120127,-0.470132,0.492552
1,-0.202145,-1.052266,-0.449204,-0.39983,2.592023,0.852042,-0.852042,0.495507,0.656577,-0.505025,...,-0.09934,-1.273240,1.273240,0.666916,-0.666916,0.606256,-0.606256,-0.120127,-0.470132,0.492552
2,-0.202145,-1.052266,2.226162,-0.39983,-0.385799,0.852042,-0.852042,0.495507,0.656577,-0.505025,...,-0.09934,0.785398,-0.785398,0.666916,-0.666916,0.606256,-0.606256,-0.120127,2.127061,-2.030242
3,-0.202145,0.950330,-0.449204,-0.39983,-0.385799,-1.173652,1.173652,0.495507,0.656577,-0.505025,...,-0.09934,0.785398,-0.785398,0.666916,-0.666916,0.606256,-0.606256,-0.120127,-0.470132,0.492552
4,-0.202145,0.950330,-0.449204,-0.39983,-0.385799,0.852042,-0.852042,0.495507,0.656577,-0.505025,...,-0.09934,-1.273240,1.273240,0.666916,-0.666916,0.606256,-0.606256,-0.120127,-0.470132,0.492552


In [11]:
scaled_vals = scaler.transform(x_accident_test)
x_testset = pd.DataFrame(scaled_vals, columns=x_accident_test.columns)

x_testset.head()

,first_point_of_impact_0,first_point_of_impact_1,first_point_of_impact_2,first_point_of_impact_3,first_point_of_impact_4,urban_or_rural_area_1,urban_or_rural_area_2,vehicle_leaving_carriageway_0,light_conditions_1,light_conditions_4,...,junction_location_9,sex_of_casualty_1,sex_of_casualty_2,sex_of_driver_1,sex_of_driver_2,did_police_officer_attend_scene_of_accident_1,did_police_officer_attend_scene_of_accident_2,casualty_severity_1,casualty_severity_2,casualty_severity_3
0,-0.202145,-1.052266,-0.449204,-0.39983,2.592023,0.852042,-0.852042,0.495507,0.656577,-0.505025,...,-0.09934,0.785398,-0.785398,0.666916,-0.666916,0.606256,-0.606256,-0.120127,-0.470132,0.492552
1,-0.202145,0.950330,-0.449204,-0.39983,-0.385799,-1.173652,1.173652,-2.018135,-1.523049,-0.505025,...,-0.09934,0.785398,-0.785398,0.666916,-0.666916,0.606256,-0.606256,-0.120127,-0.470132,0.492552
2,-0.202145,0.950330,-0.449204,-0.39983,-0.385799,0.852042,-0.852042,0.495507,0.656577,-0.505025,...,-0.09934,0.785398,-0.785398,0.666916,-0.666916,0.606256,-0.606256,-0.120127,-0.470132,0.492552
3,-0.202145,0.950330,-0.449204,-0.39983,-0.385799,-1.173652,1.173652,0.495507,0.656577,-0.505025,...,-0.09934,0.785398,-0.785398,0.666916,-0.666916,0.606256,-0.606256,-0.120127,2.127061,-2.030242
4,-0.202145,-1.052266,2.226162,-0.39983,-0.385799,0.852042,-0.852042,0.495507,0.656577,-0.505025,...,-0.09934,0.785398,-0.785398,0.666916,-0.666916,-1.649468,1.649468,-0.120127,-0.470132,0.492552


##**Setting up a Baseline:**

We will use macro-averaging in this project. For the label 3 which corresponds to "Slight", the accuracy measures will be:

Precision: ((True Positive)/((True Positive) + (False Positive))) : 48868/64674 = 0.7556

Recall: ((True Positive)/((True Positive) + (False Negative))) : 48868/48868 = 1.0

F-score: 2/(1/precision + 1/recall) = 0.86

In [12]:
print(y_accident_train.value_counts())
y_trainset.shape[0]

accident_severity
3                    48868
2                    14430
1                     1376
dtype: int64


64674

In [13]:
def naivebayes_gaussian_model(train_x_1, test_x_1, train_y_1, test_y_1):
    '''
    :param train_x_1: 70% of the train data from the observation variables
    :param test_x_1: 30% of the test data from the observation variables
    :param train_y_1: 70% of the train data from the response variables
    :param test_y_1: 30% of the test data from the response variables
    :return: model_1
    '''
    model_1 = GaussianNB()
    model_1.fit(train_x_1, train_y_1)
    print("Naive Bayes Gaussian model\n")
    print("Train accuracy:", np.round(accuracy_score(train_y_1,
                                                     model_1.predict(train_x_1)), 2)) 
    print("Train f1_score:", np.round(f1_score(train_y_1,
                                               model_1.predict(train_x_1),
                                               average ="macro"), 2))
    return model_1


def decision_tree_model(train_x_2, test_x_2, train_y_2, test_y_2):
    '''
    :param train_x_2: 70% of the train data from the observation variables
    :param test_x_2: 30% of the test data from the observation variables
    :param train_y_2: 70% of the train data from the response variables
    :param test_y_2: 30% of the test data from the response variables
    :return: model_2
    '''
    model_2 = DecisionTreeClassifier()
    model_2.fit(train_x_2, train_y_2)
    print("Decision Tree model\n")
    print("Train accuracy:", np.round(accuracy_score(train_y_2,
                                                     model_2.predict(train_x_2)), 2))
    print("Train f1_score:", np.round(f1_score(train_y_2,
                                               model_2.predict(train_x_2),
                                               average ="macro"), 2))
    return model_2


def neural_network_model(train_x_3, test_x_3, train_y_3, test_y_3):
    '''
    :param train_x_3: 70% of the train data from the observation variables
    :param test_x_3: 30% of the test data from the observation variables
    :param train_y_3: 70% of the train data from the response variables
    :param test_y_3: 30% of the test data from the response variables
    :return: model_3
    '''
    model_3 = MLPClassifier(max_iter=2500, random_state=42)
    model_3.fit(train_x_3, train_y_3)
    print("Neural network model\n")
    print("Train accuracy:", np.round(accuracy_score(train_y_3,
                                                     model_3.predict(train_x_3)), 2))
    print("Train f1_score:", np.round(f1_score(train_y_3,
                                               model_3.predict(train_x_3),
                                               average ="macro"), 2))
    return model_3


def random_forest_model( train_x_4, test_x_4, train_y_4, test_y_4):
    '''
    :param train_x_4: 70% of the train data from the observation variables
    :param test_x_4: 30% of the test data from the observation variables
    :param train_y_4: 70% of the train data from the response variables
    :param test_y_4: 30% of the test data from the response variables
    :return: model_4
    '''
    model_4 = RandomForestClassifier(random_state=42)
    model_4.fit(train_x_4, train_y_4)
    print("Random Forest model\n")
    print("Train accuracy:", np.round(accuracy_score(train_y_4,
                                                     model_4.predict(train_x_4)), 2))
    print("Train f1_score:", np.round(f1_score(train_y_4,
                                               model_4.predict(train_x_4),
                                               average ="macro"), 2))
    return model_4

##**Evaluating each model on the train data:**

##**Naive Bayes Gaussian model**

Naive Bayes is a classification algorithm for binary (two-class) and multiclass classification problems. It is called Naive Bayes or idiot Bayes because the calculations of the probabilities for each class are simplified to make their calculations tractable. We will be using this model to fit against the training dataset and observe its accuracy and f1 score.

In [14]:
# Naive Bayes Gaussian model
model_nb = naivebayes_gaussian_model(x_trainset, x_testset, y_trainset,
                                     y_testset)

Naive Bayes Gaussian model

Train accuracy: 0.72
Train f1_score: 0.4


##**Decision Tree model**
Decision Trees are a type of Supervised Machine Learning, where the data is continuously split according to a certain parameter. The tree can be explained by two entities, namely decision nodes and leaves. The leaves are the decisions or the final outcomes. And the decision nodes are where the data is split. We will be using this model to fit against the training dataset and observe its accuracy and f1 score.

In [15]:
# Decision Tree model
model_dt = decision_tree_model(x_trainset, x_testset, y_trainset,
                               y_testset)

Decision Tree model

Train accuracy: 0.97
Train f1_score: 0.95


##**MLPClassifier**
MLPClassifier stands for Multi-layer Perceptron classifier which in the name itself connects to a Neural Network. Unlike other classification algorithms such as Support Vectors or Naive Bayes Classifier, MLPClassifier relies on an underlying Neural Network to perform the task of classification. We will be using this model to fit against the training dataset and observe its accuracy and f1 score.

In [16]:
# Neural network model
model_nn = neural_network_model(x_trainset, x_testset, y_trainset,
                                y_testset)

Neural network model

Train accuracy: 0.96
Train f1_score: 0.91


##**Random Forest model**
Random forest is a Supervised Machine Learning Algorithm that is used widely in Classification problems. It builds decision trees on different samples and takes their majority vote for classification.

One of the most important features of the Random Forest Algorithm is that it can handle categorical variables in the case of classification. We will be using this model to fit against the training dataset and observe its accuracy and f1 score.

In [17]:
# Random Forest model
model_rf = random_forest_model(x_trainset, x_testset, y_trainset,
                               y_testset)

Random Forest model

Train accuracy: 0.97
Train f1_score: 0.95


##**Best model for the train set is observed for Random Forest and Decision Tree:** 
Selecting the model with best accuracy and f1 score against the train dataset.Random forest is a personal favourite over decision tree . Random forest model happens to have multiple decision trees in it. Which makes it a better tree model when compared with decision tree. Hence checking accuracy and f1score of the test dataset as well.

In [18]:
predict_y = model_rf.predict(x_testset)
print("Test accuracy:", np.round(accuracy_score(y_testset, predict_y), 2),"\n")
print("Test f1_score:", np.round(f1_score(y_testset, predict_y,
                                          average ="macro"), 2),"\n")

Test accuracy: 0.94 

Test f1_score: 0.88 



##**Hyperparameter tuning:**
The accuracy and f1score for train dataset are 0.97 and 0.95 respectively. Whereas their accuracy and f1score for test dataset are 0.94 and 0.88 respectively. Which demonstrates there is a slight overfitting. Trying hyperparameter tuning to observe if there is anyway to improve the model performance. Giving a cross validation of 5 folds and providing multiple parameters to segregate the best hyperparameter setting.

In [19]:
param_grid = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 1000],
    'max_depth': [3, 5, 10, 15],
    'min_samples_split': [3, 5, 10]
}
best_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(best_model, param_grid, cv=5, scoring='f1_macro',
                           return_train_score=True)
grid_search.fit(x_trainset, y_trainset)
print("Random Forest model\n")
print("Best esimator:{}".format(grid_search.best_estimator_))
print("Best score:{}".format(grid_search.best_score_))
cv_results = pd.DataFrame(grid_search.cv_results_)[['params', 
                                                    'mean_train_score', 
                                                    'mean_test_score']]
cv_results["diff, %"] = 100*(cv_results["mean_train_score"]
                             -cv_results["mean_test_score"])/cv_results["mean_train_score"]
print(cv_results.sort_values('mean_test_score', ascending=False))

Random Forest model

Best esimator:RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=10,
                       random_state=42)
Best score:0.8855714486959245
                                               params  mean_train_score  \
63  {'max_depth': 15, 'min_samples_split': 3, 'n_e...          0.893447   
68  {'max_depth': 15, 'min_samples_split': 3, 'n_e...          0.889797   
70  {'max_depth': 15, 'min_samples_split': 5, 'n_e...          0.888989   
69  {'max_depth': 15, 'min_samples_split': 3, 'n_e...          0.889489   
64  {'max_depth': 15, 'min_samples_split': 3, 'n_e...          0.890598   
..                                                ...               ...   
16  {'max_depth': 3, 'min_samples_split': 10, 'n_e...          0.611553   
2   {'max_depth': 3, 'min_samples_split': 3, 'n_es...          0.611553   
14  {'max_depth': 3, 'min_samples_split': 10, 'n_e...          0.611358   
7   {'max_depth': 3, 'min_samples_split': 5, 'n_es...          0.61135

##**Storing the fitted model:**
Once the optimal setting for the model is found via Gridsearch. We dump the best estimator parameters into a file of specific format.

In [20]:
import os
from joblib import dump

# create a folder where all trained models will be kept
if not os.path.exists("models"):
    os.makedirs("models")
    
dump(grid_search.best_estimator_, 'models/rf-clf.joblib')

['models/rf-clf.joblib']

##**Loading the model built with best parameters:**
Using the module joblib to import the model

In [21]:
from joblib import load

best_rf = load("models/rf-clf.joblib")

##**Running the best settings obtained from hyperparameter tuning to observe any improvement in the performance:**

We can observe that the accuracy is increased by 1% as the value obtained is 0.95. whereas there is not much variation in the f1score from the default model setup.

In [22]:
from sklearn.metrics import precision_recall_fscore_support
yhat = best_rf.predict(x_testset)

p, r, f, s = precision_recall_fscore_support(y_testset, yhat, average ="macro")
print("Random Forest")
print(f"Precision: {p}")
print(f"Recall: {r}")
print(f"F-score: {f}")
print("Test accuracy:", np.round(accuracy_score(y_testset, yhat), 2),"\n")

Random Forest
Precision: 0.9731669321333453
Recall: 0.8202475152929319
F-score: 0.8818094190446454
Test accuracy: 0.95 



##**Conclusion:**
This individual project was successfully implemented. Several alternative models were used to address the business problem.
The baseline for the models were calculated manually and model evaluation was done against four models, out of which two models performed well. One of the two models were checked against the test dataset to identify, if there is a case of overfitting or underfitting. Slight overfitting was observed hence hyperparameter tunning was employed. Based on hyperparameter tuning, the best setting that can improve the model performance was obtained. Finally we singled out the settings to individually observe the performance.

##**Key finding:**

We are able to accurately predict accident severity based on the features that are currently selected.

##**Possible future improvements:**

Feature engineering and Dimensionality reduction were methods that were not explored in the individual task. These methods can be investigated further for future improvements. 